In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch import nn
from torch.utils.data import random_split,Dataset,DataLoader
from torchvision import models,datasets,transforms
from sklearn.metrics import f1_score
from tqdm import tqdm
import warnings 
import time
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#data dir.
data_root_dir ="/content/drive/MyDrive/multicalss_data/data/dataset/data/"
label_root_dir =  "/content/drive/MyDrive/multicalss_data/data/dataset/train.csv"
test_root_dir =  "/content/drive/MyDrive/multicalss_data/data/dataset/test.csv"

In [11]:
test_root_dir

'/content/drive/MyDrive/multicalss_data/data/dataset/test.csv'

In [6]:
# Hyperparameters.
N_classes =20
learning_rate = 0.001
epochs = 1
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [7]:
torch.cuda.is_available()

True

In [ ]:
#Label pre-processing
train_csv=pd.read_csv(label_root_dir,error_bad_lines=False)
labels=pd.DataFrame(np.zeros((len(train_csv),20), dtype=np.int), columns=np.arange(20))
for i in range(train_csv.shape[0]):
    data=list(map(int, train_csv['Labels'][i].split(" ")))
    for j in data:
        labels.iloc[i,j] = 1
train_set = pd.concat([train_csv['ImageID'], train_csv['Caption'], labels], axis=1)

test_csv=pd.read_csv(test_root_dir, error_bad_lines=False)
test_csv1 = test_csv.iloc[:6889,:]
#There is an error point here, special treatment
test_csv1 = test_csv1.append({'ImageID':'36889.jpg','Caption':''},ignore_index=True)
test_csv2 = test_csv.iloc[6889:,:]
test_csv = pd.concat([test_csv1,test_csv2],ignore_index=True)
labels_test=pd.DataFrame(np.zeros((len(test_csv),20), dtype=np.int), columns=np.arange(20))
test_set = pd.concat([test_csv['ImageID'], test_csv['Caption'], labels_test], axis=1)

b'Skipping line 9086: expected 3 fields, saw 4\nSkipping line 9510: expected 3 fields, saw 4\nSkipping line 18114: expected 3 fields, saw 4\nSkipping line 27169: expected 3 fields, saw 4\n'
b'Skipping line 6891: expected 2 fields, saw 3\n'


In [ ]:
#读取图片
class ImageDataset(Dataset):
    def __init__(self, data_root_dir, csv_file_label,test=0):
        self.data_root_dir = data_root_dir
        self.csv_file_label = csv_file_label
        if test==0:
            self.transforms = transforms.Compose([ 
                              transforms.ColorJitter(0.2,0.2,0.2,0.2), # Randomly change the brightness, contrast and saturation of the picture.
                              transforms.GaussianBlur(kernel_size=(5, 11),sigma=(5, 10.0)), # Gaussian blur.
                              transforms.RandomHorizontalFlip(p=0.5), # Flip an image horizontally.
                              transforms.RandomVerticalFlip(p=0.5), # Flip the image vertically.
                              transforms.Resize((224,224)) , 
                              transforms.ToTensor(),
                              transforms.Normalize([0.423, 0.424, 0.426], [0.190, 0.190, 0.191])])
        else:
             self.transforms = transforms.Compose([
                              transforms.Resize((224,224)) , 
                              transforms.ToTensor(),
                              transforms.Normalize([0.423, 0.424, 0.426], [0.190, 0.190, 0.191])])           
        
    def __len__(self):
        return len(self.csv_file_label)
    
    def __getitem__(self, i):
        image_dir = self.data_root_dir + self.csv_file_label.iloc[i].ImageID
        image = Image.open(image_dir).convert("RGB")
        image = self.transforms(image)
        label = self.csv_file_label.iloc[i][2:].tolist()
        label = torch.tensor(label, dtype=torch.float32)
        caption = self.csv_file_label.iloc[i]['Caption']
        return image, label, caption
        #return image, label

class ImageDataloader():
    def __init__(self, data_root_dir, csv_file_label,test=0):
        if test==0:
            train_percentage = 0.99
            image_dataset = ImageDataset(data_root_dir, csv_file_label,test)
            dataset_len = len(image_dataset)
            self.train_set, self.valid_set = random_split(image_dataset, [int(dataset_len*train_percentage), (dataset_len-int(dataset_len*train_percentage))])
        else:
            image_dataset = ImageDataset(data_root_dir, csv_file_label,test)
            dataset_len = len(image_dataset)
            self.test_set = image_dataset
            
    def make_loader(self, batch_size,test=0):
        if test==0:
            return DataLoader(self.train_set, shuffle=True, batch_size=batch_size,num_workers=1,prefetch_factor=batch_size*2), DataLoader(self.valid_set, shuffle=True, batch_size=batch_size,num_workers=1,prefetch_factor=batch_size*2)
        else:
            return DataLoader(self.test_set, shuffle=False, batch_size=batch_size,num_workers=1,prefetch_factor=batch_size*2)

In [ ]:
train_set.iloc[0]["Caption"]

'Woman in swim suit holding parasol on sunny day.'

In [ ]:
#build model.
def BuildModel(device,backbone = "vit_l_32"):
    model = models.vit_l_32(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    N_fc_in = 1024
    N_fc_layers = 3
    fc_layers = []
    for i in range(3):
        fc_layers.append(nn.Linear(N_fc_in//(2**i), N_fc_in//(2**(i+1))))
        fc_layers.append(nn.ReLU())
        fc_layers.append(nn.BatchNorm1d(N_fc_in//(2**(i+1))))
        fc_layers.append(nn.Dropout(0.2))
    fc_layers.append( nn.Linear( N_fc_in//(2**(N_fc_layers)) , N_classes ))
    model.heads = nn.Sequential(*fc_layers)
    model = model.to(device)
    return model

In [ ]:
2**(2+1)

8

In [ ]:
def accuracy_calculate(y_true, outputs_torch, curr_batch_size):
    y_true = y_true.to(torch.int).numpy()
    
    y_pred = (torch.sigmoid(outputs_torch).data > 0.5).cpu().detach().to(torch.int).numpy()
    
    return f1_score(y_true, y_pred, average='samples') * curr_batch_size

In [ ]:
# model =  torch.load('../input/modelt/modelres101.pth')

In [ ]:
model = BuildModel(device=device,backbone = "vit")

dataset_loader = ImageDataloader(data_root_dir,train_set)
train_dataloader, valid_dataloader = dataset_loader.make_loader(batch_size = 256)
len_train = len(train_dataloader.dataset)
len_valid = len(valid_dataloader.dataset)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=0.005)
criterion = nn.BCEWithLogitsLoss()

for epoch_it in range(epochs):
    print("epoch:", epoch_it, '/', epochs)
    train_accuracy = 0.0 
    train_loss = 0.0
    model.train()
    for images, labels, _ in tqdm(train_dataloader):
        torch.set_grad_enabled(True)
        images = images.to(device)
        labels_processed = labels.to(device)
        curr_batch_size = images.size(0)
        outputs = model(images)
        train_accuracy += accuracy_calculate(labels, outputs, curr_batch_size)
        loss = criterion(outputs, labels_processed)
        loss.backward()
        train_loss += curr_batch_size*loss.item()
        optimizer.step()
        optimizer.zero_grad()
    print("train loss:", (train_loss/len_train),
          "accuracy:", (train_accuracy/len_train))

    valid_accuracy = 0.0
    valid_loss = 0.0
    model.eval()
    for images, labels, _ in tqdm(valid_dataloader):
        torch.set_grad_enabled(False)
        images = images.to(device)
        labels_processed = labels.to(device)
        curr_batch_size = images.size(0)
        outputs = model(images)
        valid_accuracy += accuracy_calculate(labels, outputs, curr_batch_size)
        loss = criterion(outputs, labels_processed)
        valid_loss += curr_batch_size*loss.item()
    print("valid loss:", (valid_loss/len_valid),
          "accuracy", (valid_accuracy/len_valid))
    scheduler.step()

epoch: 0 / 1


 86%|████████▌ | 100/116 [11:09<01:43,  6.48s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62f64830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fea62f64830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_wo

train loss: 0.5350469746466341 accuracy: 0.42049467867101264


100%|██████████| 2/2 [00:10<00:00,  5.08s/it]

valid loss: 0.245620592435201 accuracy 0.6647999037999038


In [ ]:
#torch.save(model,'modelres101.pth')

In [ ]:
key_word = {}
key_word[0] = 'jkldhgdlsuhfgkvsghndjkvg'
key_word[1] = 'person'
key_word[2] = 'bicycle'
key_word[3] = 'jkldhgdlsuhfgkvsghndjkvg'
key_word[4] = 'motor'
key_word[5] = 'plane'
key_word[6] = 'buses'
key_word[7] = 'train'
key_word[8] = 'truck'
key_word[9] = 'boat'
key_word[10] = 'traffic light'
key_word[11] = 'fire hydrant'
key_word[12] = 'jkldhgdlsuhfgkvsghndjkvg'
key_word[13] = 'stop sign'
key_word[14] = 'meter'
key_word[15] = 'bench'
key_word[16] = 'bird'
key_word[17] = ' cat '
key_word[18] = ' dog '
key_word[19] = 'horse'
key_word

{0: 'jkldhgdlsuhfgkvsghndjkvg',
 1: 'person',
 2: 'bicycle',
 3: 'jkldhgdlsuhfgkvsghndjkvg',
 4: 'motor',
 5: 'plane',
 6: 'buses',
 7: 'train',
 8: 'truck',
 9: 'boat',
 10: 'traffic light',
 11: 'fire hydrant',
 12: 'jkldhgdlsuhfgkvsghndjkvg',
 13: 'stop sign',
 14: 'meter',
 15: 'bench',
 16: 'bird',
 17: ' cat ',
 18: ' dog ',
 19: 'horse'}

In [ ]:
testset_loader = ImageDataloader(data_root_dir,test_set,1)
test_dataloader = testset_loader.make_loader(1,1)
test_label_list=[]
res = test_set
for images, labels,caption in tqdm(test_dataloader):
    torch.set_grad_enabled(False)
    images = images.to(device)
    outputs = model(images)
    outputs = outputs.cpu().numpy()
    outputs = (outputs>0.5)
    data = ""
    for i in range(len(outputs[0])):
        if outputs[0][i]:
            if data=="":
                data += str(i)
            else:
                data+= " "
                data+=str(i)
        else:
            if key_word[int(i)] in caption[0].lower():
                print(i,caption)
                if data=="":
                    data += str(i)
                else:
                    data+= " "
                    data+=str(i)
            else:
                if i==3:
                    if ('cars' in caption[0].lower()) and ('train car' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                    elif (' car ' in caption[0].lower()) and ('train car' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                    elif (' car.' in caption[0].lower()) and ('train car' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==2:
                    if ('bike' in caption[0].lower()) and ('motor' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==17:
                    if (' cat.' in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==18:
                    if (' dog.' in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==5:
                    if ('aircraft' in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==9:
                    if ('ships' in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                if i==6:
                    if (' bus ' in caption[0].lower()) and ('bus stop' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
                    elif (' bus.' in caption[0].lower()) and ('bus stop' not in caption[0].lower()):
                        print(i,caption)
                        if data=="":
                            data += str(i)
                        else:
                            data+= " "
                            data+=str(i)
    test_label_list.append(data)
res['Labels'] = test_label_list
res.to_csv("res.csv",index=False)
print(res)

  0%|          | 12/10000 [00:00<05:54, 28.15it/s]

11 ('A pink fire hydrant in the middle of a yard.',)
9 ('A small boat floating on top of a body of water.',)
9 ('A body of water with a sailboat in the background.',)
1 ('A person riding a horse next to a big black and brown dog.',)
18 ('A person riding a horse next to a big black and brown dog.',)
19 ('A person riding a horse next to a big black and brown dog.',)


  0%|          | 23/10000 [00:00<04:57, 33.50it/s]

8 ('A red truck driving along a mountainous countryside.',)
17 ('A cat is sitting on top of a microwave under a cabinet.',)


  0%|          | 35/10000 [00:01<04:43, 35.20it/s]

...

15 ('two men sitting on a bench wearing hats and suits',)
18 ('A small dog is carrying a teddy bear in its mouth',)
1 ('A truck sitting in front of a person standing on a  loading platform.',)
8 ('A truck sitting in front of a person standing on a  loading platform.',)


100%|█████████▉| 9997/10000 [05:05<00:00, 31.20it/s]

2 ('A man skateboards down the street while a woman bikes.',)


100%|██████████| 10000/10000 [05:06<00:00, 32.67it/s]


        ImageID                                            Caption  0  1  2  \
0     30000.jpg  A little girl waring a krispy kreme hat holdin...  0  0  0   
1     30001.jpg  A beautiful young woman holding an orange fris...  0  0  0   
2     30002.jpg  A group of people sitting on couch next to a c...  0  0  0   
3     30003.jpg         A person on a snowboard rides on the hill.  0  0  0   
4     30004.jpg  A man riding a skateboard with a helmet on in ...  0  0  0   
...         ...                                                ... .. .. ..   
9995  39995.jpg  A group of men riding surfboards riding a mass...  0  0  0   
9996  39996.jpg  A motorcycle parked next to a car in a parking...  0  0  0   
9997  39997.jpg            a little boy that is playing with a wii  0  0  0   
9998  39998.jpg  group of kids play Frisbee golf in the middle ...  0  0  0   
9999  39999.jpg   A man in a gray jacket standing next to a woman.  0  0  0   

      3  4  5  6  7  ...  11  12  13  14  15  16  1